# Imports

In [1]:
import numpy as np
from functools import partial

In [2]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [3]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.backends import cudnn

In [4]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [5]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model

In [6]:
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from sacred import Ingredient

In [7]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset
from utils.metrics import *

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ranx/qrels_run_common.py:7: UserWarning: Sorting disabled. Assumes that you provided sorted doc_ids!
  warnings.warn("Sorting disabled. Assumes that you provided sorted doc_ids!")


In [8]:
from utils import pickle_load
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [9]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [10]:
feature_name = 'r50_gldv1'
set_name = 'tuto'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [11]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [12]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() and not cpu else 'cpu')
torch.manual_seed(seed)

In [14]:
if cudnn_flag == 'deterministic':
    setattr(cudnn, cudnn_flag, True)

In [15]:
def read_file(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [16]:
def get_sets(desc_name, 
        train_data_dir, test_data_dir, 
        train_txt, test_txt, test_gnd_file, 
        max_sequence_len=500):
    ####################################################################################################################################
    train_lines     = read_file(osp.join(train_data_dir, train_txt))
    train_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in train_lines]
    train_set       = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    query_train_set = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    ####################################################################################################################################
    test_gnd_data = None if test_gnd_file is None else pickle_load(osp.join(test_data_dir, test_gnd_file))
    query_lines   = read_file(osp.join(test_data_dir, test_txt[0]))
    gallery_lines = read_file(osp.join(test_data_dir, test_txt[1]))
    query_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in query_lines]
    gallery_samples = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in gallery_lines]
    gallery_set = FeatureDataset(test_data_dir, gallery_samples, desc_name, max_sequence_len)
    query_set   = FeatureDataset(test_data_dir, query_samples,   desc_name, max_sequence_len, gnd_data=test_gnd_data)
        
    return (train_set, query_train_set), (query_set, gallery_set)

In [17]:
class MetricLoaders(NamedTuple):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    gallery: Optional[DataLoader] = None

In [18]:
def get_loaders(desc_name, train_data_dir, 
    batch_size=8, test_batch_size=8, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100):

    (train_set, query_train_set), (query_set, gallery_set) = get_sets(desc_name, 
        train_data_dir=train_data_dir,
        test_data_dir=train_data_dir,
        train_txt='tuto_query.txt',
        test_txt=('tuto_query.txt', 'tuto_selection.txt'),
        test_gnd_file='gnd_tuto.pkl', 
        max_sequence_len=500)

    if sampler == 'random':
        train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)
    elif sampler == 'triplet':
        train_nn_inds = osp.join(train_data_dir, set_name+'_nn_inds_%s.pkl'%desc_name)
        train_sampler = TripletSampler(train_set.targets, batch_size, train_nn_inds, num_candidates)
    else:
        raise ValueError('Invalid choice of sampler ({}).'.format(sampler))
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
        
    query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
    gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

    return MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories)), recalls

In [19]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [20]:
#train_txt = set_name+'_query.txt'
#query_lines     = read_file(osp.join(train_data_dir, train_txt))
#train_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in train_lines]


In [21]:
#test_txt = ('test_query.txt', 'test_selection.txt')
#desc_name = 'r50_gldv1'
#query_lines   = read_file(osp.join(train_data_dir, test_txt[0]))
#gallery_lines = read_file(osp.join(train_data_dir, test_txt[1]))
#query_samples   = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in query_lines]
#gallery_samples = [(line.split(';;')[0], int(line.split(';;')[1]), int(line.split(';;')[2]), int(line.split(';;')[3])) for line in gallery_lines]
#gallery_set = FeatureDataset(train_data_dir, gallery_samples, desc_name, max_sequence_len)
#query_set   = FeatureDataset(train_data_dir, query_samples,   desc_name, max_sequence_len)

In [22]:
#gallery_loader

In [23]:
(train_set, query_train_set), (query_set, gallery_set) = get_sets('r50_gldv1',
            train_data_dir,
            train_data_dir,
            set_name+'_query.txt',
            (set_name+'_query.txt',set_name+'_selection.txt'),
            'gnd_'+set_name+'.pkl',
            500)

In [24]:
batch_size      = 1
test_batch_size = 1
max_sequence_len = 100
sampler = 'random'
if sampler == 'random':
   train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)

In [25]:
num_workers = 8  # number of workers used ot load the data
pin_memory  = True  # use the pin_memory option of DataLoader 
num_candidates = 100
recalls = [1, 5, 6, 10]

In [26]:
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [27]:
query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [28]:
loaders, recall_ks = get_loaders(desc_name='r50_gldv1',
    train_data_dir='/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt', 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100)

In [29]:
model_ingredient = Ingredient('model', interactive=True)

In [30]:
def get_model(num_global_features, num_local_features, seq_len, dim_K, dim_feedforward, nhead, num_encoder_layers, dropout, activation, normalize_before):
    return MatchERT(d_global=num_global_features, d_model=num_local_features, seq_len=seq_len, d_K=dim_K, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, normalize_before=normalize_before)

In [31]:
name = 'rrt'
num_global_features = 2048  
num_local_features = 128  
seq_len = 1004
dim_K = 256
dim_feedforward = 1024
nhead = 4
num_encoder_layers = 6
dropout = 0.0 
activation = "relu"
normalize_before = False

In [32]:
model = get_model(num_global_features,num_local_features,seq_len,dim_K,dim_feedforward,nhead,num_encoder_layers,dropout,activation,normalize_before)

In [33]:
if resume is not None:
   checkpoint = torch.load(resume, map_location=torch.device('cpu'))
   model.load_state_dict(checkpoint['state'], strict=True)

In [34]:
model.to(device)
model.eval()

MatchERT(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Lin

In [35]:
loaders.query.dataset.desc_name, loaders.query.dataset.desc_name, loaders.query.dataset.data_dir

('r50_gldv1',
 'r50_gldv1',
 '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt')

In [36]:
loaders.query

In [37]:
nn_inds_path = osp.join(loaders.query.dataset.data_dir, set_name+'_nn_inds_%s.pkl'%loaders.query.dataset.desc_name)
cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()

In [38]:
cache_nn_inds.size()

torch.Size([100, 100])

In [39]:
num_samples, top_k = cache_nn_inds.size()
top_k = min(top_k, 100)
top_k

100

In [40]:
def evaluate(
        model: nn.Module,
        cache_nn_inds: torch.Tensor,
        query_loader: DataLoader,
        gallery_loader: DataLoader,
        recall: List[int]):
    model.eval()
    device = next(model.parameters()).device
    to_device = lambda x: x.to(device, non_blocking=True)

    query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

    with torch.no_grad():
        for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
            q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
            query_global.append(q_global.cpu())
            query_local.append(q_local.cpu())
            query_mask.append(q_mask.cpu())
            query_scales.append(q_scales.cpu())
            query_positions.append(q_positions.cpu())
            query_names.extend(list(q_names))

        query_global    = torch.cat(query_global, 0)
        query_local     = torch.cat(query_local, 0)
        query_mask      = torch.cat(query_mask, 0)
        query_scales    = torch.cat(query_scales, 0)
        query_positions = torch.cat(query_positions, 0)

        for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
            g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
            gallery_global.append(g_global.cpu())
            gallery_local.append(g_local.cpu())
            gallery_mask.append(g_mask.cpu())
            gallery_scales.append(g_scales.cpu())
            gallery_positions.append(g_positions.cpu())
            gallery_names.extend(list(g_names))
            
        gallery_global    = torch.cat(gallery_global, 0)
        gallery_local     = torch.cat(gallery_local, 0)
        gallery_mask      = torch.cat(gallery_mask, 0)
        gallery_scales    = torch.cat(gallery_scales, 0)
        gallery_positions = torch.cat(gallery_positions, 0)

        torch.cuda.empty_cache()
        evaluate_function = partial(mean_average_precision_revisited_rerank, model=model, cache_nn_inds=cache_nn_inds,
            query_global=query_global, query_local=query_local, query_mask=query_mask, query_scales=query_scales, query_positions=query_positions, 
            gallery_global=gallery_global, gallery_local=gallery_local, gallery_mask=gallery_mask, gallery_scales=gallery_scales, gallery_positions=gallery_positions, 
            ks=recall, 
            gnd=query_loader.dataset.gnd_data,
        )
        metrics = evaluate_function()
    return metrics 

In [41]:
model.eval()
device = next(model.parameters()).device
to_device = lambda x: x.to(device, non_blocking=True)

query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

with torch.no_grad():
    for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
        q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
        query_global.append(q_global.cpu())
        query_local.append(q_local.cpu())
        query_mask.append(q_mask.cpu())
        query_scales.append(q_scales.cpu())
        query_positions.append(q_positions.cpu())
        query_names.extend(list(q_names))

    query_global    = torch.cat(query_global, 0)
    query_local     = torch.cat(query_local, 0)
    query_mask      = torch.cat(query_mask, 0)
    query_scales    = torch.cat(query_scales, 0)
    query_positions = torch.cat(query_positions, 0)
    
    for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
        g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
        gallery_global.append(g_global.cpu())
        gallery_local.append(g_local.cpu())
        gallery_mask.append(g_mask.cpu())
        gallery_scales.append(g_scales.cpu())
        gallery_positions.append(g_positions.cpu())
        gallery_names.extend(list(g_names))

    gallery_global    = torch.cat(gallery_global, 0)
    gallery_local     = torch.cat(gallery_local, 0)
    gallery_mask      = torch.cat(gallery_mask, 0)
    gallery_scales    = torch.cat(gallery_scales, 0)
    gallery_positions = torch.cat(gallery_positions, 0)

In [42]:
torch.cuda.empty_cache()

In [48]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
f, r, a

(3602432, 39845888, 36243456)

In [46]:
device = next(model.parameters()).device
query_global    = query_global.to(device)
query_local     = query_local.to(device)
query_mask      = query_mask.to(device)
query_scales    = query_scales.to(device)
query_positions = query_positions.to(device)

In [47]:
num_samples, top_k = cache_nn_inds.size()
top_k = min(100, top_k)

In [49]:
print("top_k: ", top_k)    
print("query_global size: ", query_global.shape)
print("query_local size: ", query_local.shape)
print("query_mask size: ", query_mask.shape)
print("query_scales size: ", query_scales.shape)
print("query_positions size: ", query_positions.shape)

top_k:  100
query_global size:  torch.Size([100, 2048])
query_local size:  torch.Size([100, 500, 128])
query_mask size:  torch.Size([100, 500])
query_scales size:  torch.Size([100, 500])
query_positions size:  torch.Size([100, 500, 2])


In [50]:
print("top_k: ", top_k)    
print("gallery_global size: ", gallery_global.shape)
print("gallery_local size: ", gallery_local.shape)
print("gallery_mask size: ", gallery_mask.shape)
print("gallery_scales size: ", gallery_scales.shape)
print("gallery_positions size: ", gallery_positions.shape)

top_k:  100
gallery_global size:  torch.Size([10000, 2048])
gallery_local size:  torch.Size([10000, 500, 128])
gallery_mask size:  torch.Size([10000, 500])
gallery_scales size:  torch.Size([10000, 500])
gallery_positions size:  torch.Size([10000, 500, 2])


In [51]:
gallery_global = gallery_global.view(query_global.size(dim=0), 100, query_global.size(dim=1))
gallery_local = gallery_local.view(query_local.size(dim=0), 100, query_local.size(dim=1), query_local.size(dim=2))
gallery_mask = gallery_mask.view(query_mask.size(dim=0), 100, query_mask.size(dim=1))
gallery_scales = gallery_scales.view(query_scales.size(dim=0), 100, query_scales.size(dim=1))
gallery_positions = gallery_positions.view(query_positions.size(dim=0), 100, query_positions.size(dim=1), query_positions.size(dim=2))

In [52]:
print("top_k: ", top_k)    
print("gallery_global size: ", gallery_global.shape)
print("gallery_local size: ", gallery_local.shape)
print("gallery_mask size: ", gallery_mask.shape)
print("gallery_scales size: ", gallery_scales.shape)
print("gallery_positions size: ", gallery_positions.shape)

top_k:  100
gallery_global size:  torch.Size([100, 100, 2048])
gallery_local size:  torch.Size([100, 100, 500, 128])
gallery_mask size:  torch.Size([100, 100, 500])
gallery_scales size:  torch.Size([100, 100, 500])
gallery_positions size:  torch.Size([100, 100, 500, 2])


In [54]:
torch.cuda.memory_summary(device='cuda:0', abbreviated=True)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   35394 KB |   35394 KB |   35394 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |   35394 KB |   35394 KB |   35394 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |   38912 KB |   38912 KB |   38912 KB |       0 B  |\n|--------------------------------------------------

In [55]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [56]:
eval_nn_inds = deepcopy(cache_nn_inds.cpu().data.numpy())
# Exclude the junk images as in DELG (https://github.com/tensorflow/models/blob/44cad43aadff9dd12b00d4526830f7ea0796c047/research/delf/delf/python/detect_to_retrieve/image_reranking.py#L190)
for i in range(num_samples):
    junk_ids = gnd_data['gnd'][i]['junk']
    all_ids = eval_nn_inds[i]
    pos = np.in1d(all_ids, junk_ids)
    neg = np.array([not x for x in pos])
    new_ids = np.concatenate([np.arange(len(all_ids))[neg], np.arange(len(all_ids))[pos]])
    new_ids = all_ids[new_ids]
    eval_nn_inds[i] = new_ids
eval_nn_inds = torch.from_numpy(eval_nn_inds)

In [57]:
eval_nn_inds.shape

torch.Size([100, 100])

In [58]:
i = 0
nnids = eval_nn_inds[:, i]
nnids, nnids.shape, gallery_global.shape

(tensor([78, 97, 96,  0,  8, 95, 43, 98, 17, 43, 88, 63, 24, 79, 76, 50,  8, 40,
         67, 81, 95, 26, 69, 70,  0, 71, 30, 56, 20, 17, 84,  7,  5, 73, 61, 78,
         51,  1, 36, 99, 46, 20,  0, 65, 48, 60, 29, 43, 40, 63, 34, 20,  9, 74,
         65, 93, 23, 76, 48, 94, 22,  2,  8, 46, 98, 47, 22, 64, 98, 29, 92, 57,
         58, 90, 69, 30, 40, 13,  6, 15, 58, 58, 62, 59, 68, 76,  4, 67, 52, 16,
         94, 21, 76, 94,  1, 82, 82, 73, 65, 69]),
 torch.Size([100]),
 torch.Size([100, 100, 2048]))

In [71]:
import gc

In [73]:
torch.cuda.empty_cache()
gc.collect()

20

In [80]:
query_global[0].unsqueeze(dim=0).shape, nnids.size(dim=0), nnids.size()

(torch.Size([1, 2048]), 100, torch.Size([100]))

In [86]:
c_scores =  []
for iterator in range(100):
    index_global = [gallery_global[iterator, nnids[iterator]]]
    index_local = [gallery_local[iterator, nnids[iterator]]]
    index_mask = [gallery_mask[iterator, nnids[iterator]]]
    index_scales = [gallery_scales[iterator, nnids[iterator]]]
    index_positions = [gallery_positions[iterator, nnids[iterator]]]

    torch.cuda.empty_cache()
    index_global = torch.from_numpy(np.stack(index_global, axis=0))

    torch.cuda.empty_cache()
    index_local = torch.from_numpy(np.stack(index_local, axis=0))

    torch.cuda.empty_cache()
    index_mask = torch.from_numpy(np.stack(index_mask, axis=0))

    torch.cuda.empty_cache()
    index_scales = torch.from_numpy(np.stack(index_scales, axis=0))

    torch.cuda.empty_cache()
    index_positions = torch.from_numpy(np.stack(index_positions, axis=0))

    q_global = query_global[iterator].unsqueeze(dim=0)
    q_local = query_local[iterator].unsqueeze(dim=0)
    q_mask = query_mask[iterator].unsqueeze(dim=0)
    q_scales = query_scales[iterator].unsqueeze(dim=0)
    q_postions = query_positions[iterator].unsqueeze(dim=0)

    iter_scores = model(
        q_global, q_local, q_mask, q_scales, q_positions,
        index_global.to(device),
        index_local.to(device),
        index_mask.to(device),
        index_scales.to(device),
        index_positions.to(device))
    
    c_scores.append(iter_scores.cpu().data)

In [125]:
scores

tensor([[ 1.7295,  1.7295,  1.7295,  ..., -1.7395, -1.7395, -1.7395],
        [ 1.4293, -0.2676,  0.2743,  ..., -1.9606, -1.9606, -3.6145],
        [ 1.4293, -0.2676,  0.2743,  ..., -3.2511, -3.3540, -3.5074],
        ...,
        [-1.0915, -3.0135, -3.2550,  ..., -2.9478, -2.9478, -2.9478],
        [-3.2819, -2.7184, -3.0668,  ..., -5.5008, -2.9316, -2.9316],
        [-2.6162, -1.0304, -1.1477,  ..., -2.9478, -2.9478, -5.2138]])

In [120]:
scores = []
for i in tqdm(range(top_k)):
    nnids = eval_nn_inds[:, i]
    topk_scores =  []
    for iterator in range(nnids.size(dim=0)):
        index_global = [gallery_global[iterator, nnids[iterator]]]
        index_local = [gallery_local[iterator, nnids[iterator]]]
        index_mask = [gallery_mask[iterator, nnids[iterator]]]
        index_scales = [gallery_scales[iterator, nnids[iterator]]]
        index_positions = [gallery_positions[iterator, nnids[iterator]]]

        torch.cuda.empty_cache()
        index_global = torch.from_numpy(np.stack(index_global, axis=0))

        torch.cuda.empty_cache()
        index_local = torch.from_numpy(np.stack(index_local, axis=0))

        torch.cuda.empty_cache()
        index_mask = torch.from_numpy(np.stack(index_mask, axis=0))

        torch.cuda.empty_cache()
        index_scales = torch.from_numpy(np.stack(index_scales, axis=0))

        torch.cuda.empty_cache()
        index_positions = torch.from_numpy(np.stack(index_positions, axis=0))
        
        q_global = query_global[iterator].unsqueeze(dim=0)
        q_local = query_local[iterator].unsqueeze(dim=0)
        q_mask = query_mask[iterator].unsqueeze(dim=0)
        q_scales = query_scales[iterator].unsqueeze(dim=0)
        q_postions = query_positions[iterator].unsqueeze(dim=0)
        
        iter_scores = model(
            q_global, q_local, q_mask, q_scales, q_positions,
            index_global.to(device),
            index_local.to(device),
            index_mask.to(device),
            index_scales.to(device),
            index_positions.to(device))
        
        topk_scores.append(iter_scores.cpu().data)
    
    current_scores = torch.from_numpy(np.stack(topk_scores, axis=0)).squeeze(1)

    torch.cuda.empty_cache()
    scores.append(current_scores.cpu().data)

100%|██████████| 100/100 [02:03<00:00,  1.23s/it]


In [121]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [122]:
len(scores[3])

100

In [123]:
scores = torch.stack(scores, -1) # nb_queries x 100
closest_dists, indices = torch.sort(scores, dim=-1, descending=True)
closest_indices = torch.gather(eval_nn_inds, -1, indices)
ranks = deepcopy(eval_nn_inds)
ranks[:, :top_k] = deepcopy(closest_indices)
ranks = ranks.cpu().data.numpy().T
# pickle_save('eval_nn_inds.pkl', ranks.T)
out = compute_metrics('viquae', ranks, gnd_data['gnd'], kappas=[1,5,10])

{'map': 18.69, 'map@1': 2.34, 'map@5': 5.6, 'map@10': 8.04, 'mrr': 23.29, 'mrr@1': 15.0, 'mrr@5': 20.83, 'mrr@10': 21.95, 'precision': 9.71, 'precision@1': 15.0, 'precision@5': 14.4, 'precision@10': 14.4}


In [74]:
iter_scores.cpu().data.item()

1.4292610883712769

In [97]:
torch.from_numpy(np.stack(c_scores, axis=0)).squeeze(1), c_scores

(tensor([ 1.7295,  1.4293,  1.4293, -0.3602, -1.9409,  7.8809,  4.1849, -1.1213,
         -2.7787, -3.7398,  2.6100, -3.0069, -4.3349,  1.2292,  7.7114,  6.0259,
         -0.4281,  2.0132, -3.4566,  0.3535, -1.6808,  0.4280,  0.2461, -2.6675,
         -2.9625,  4.2006, -1.7271,  2.2924, -0.5409, -0.9304,  0.5119, -1.0696,
          7.2333, -2.6997, -0.6677, -3.1610,  8.0865,  1.6478, -1.4728,  1.0965,
          3.3788,  7.8428,  5.6013,  0.2370,  6.0813, -0.4101, -0.0607,  1.5350,
         -2.1898, -0.3205, -2.9534, -2.8657, -2.0753,  8.4222,  8.4184,  0.2731,
          0.5053,  8.4222, -2.7350,  6.6646, -2.2468,  8.4184, -1.1454, -3.0600,
         -1.1788,  5.0812, -1.3574,  6.1592,  7.6116,  8.1979, -3.5890,  4.4803,
         -1.6283,  3.0267, -3.4154, -3.0453, -1.8392,  0.1695,  0.3817,  1.0198,
          8.4204, -3.2018, -1.7226, -2.7281,  4.0545,  0.6906, -1.9036,  2.2423,
          3.2200,  7.0380,  0.8479,  8.4142,  0.2687, -2.6870,  8.4194,  2.1687,
         -3.2969, -1.0915, -

In [95]:
topk_scores = []
for i in tqdm(range(top_k)):
    nnids = eval_nn_inds[:, i]
    #iter_scores     = []
    for iterator in range(nnids.size(dim=0)):
        index_global = [gallery_global[iterator, nnids[iterator]]]
        index_local = [gallery_local[iterator, nnids[iterator]]]
        index_mask = [gallery_mask[iterator, nnids[iterator]]]
        index_scales = [gallery_scales[iterator, nnids[iterator]]]
        index_positions = [gallery_positions[iterator, nnids[iterator]]]

        torch.cuda.empty_cache()
        index_global = torch.from_numpy(np.stack(index_global, axis=0))

        torch.cuda.empty_cache()
        index_local = torch.from_numpy(np.stack(index_local, axis=0))

        torch.cuda.empty_cache()
        index_mask = torch.from_numpy(np.stack(index_mask, axis=0))

        torch.cuda.empty_cache()
        index_scales = torch.from_numpy(np.stack(index_scales, axis=0))

        torch.cuda.empty_cache()
        index_positions = torch.from_numpy(np.stack(index_positions, axis=0))

        print("index_global size: ", index_global.shape)
        print("index_local size: ", index_local.shape)
        print("index_mask size: ", index_mask.shape)
        print("index_scales size: ", index_scales.shape)
        print("index_positions size: ", index_positions.shape)


        print("index_global device: ", index_global.get_device())
        print("index_local device: ", index_local.get_device())
        print("index_mask device: ", index_mask.get_device())
        print("index_scales device: ", index_scales.get_device())
        print("index_positions device: ", index_positions.get_device())
        
        q_global = query_global[iterator].unsqueeze(dim=0)
        q_local = query_local[iterator].unsqueeze(dim=0)
        q_mask = query_mask[iterator].unsqueeze(dim=0)
        q_scales = query_scales[iterator].unsqueeze(dim=0)
        q_postions = query_positions[iterator].unsqueeze(dim=0)
        
        print("query_global size: ", q_global.shape)
        print("query_local size: ", q_local.shape)
        print("query_mask size: ", q_mask.shape)
        print("query_scales size: ", q_scales.shape)
        print("query_positions size: ", q_positions.shape)
        
        iter_scores = model(
            q_global, q_local, q_mask, q_scales, q_positions,
            index_global.to(device),
            index_local.to(device),
            index_mask.to(device),
            index_scales.to(device),
            index_positions.to(device))
    
    torch.from_numpy(np.stack(c_scores, axis=0)).squeeze(1)

#    torch.cuda.empty_cache()
#    scores.append(current_scores.cpu().data)

  0%|          | 0/100 [00:00<?, ?it/s]

index_global size:  torch.Size([1, 2048])
index_local size:  torch.Size([1, 500, 128])
index_mask size:  torch.Size([1, 500])
index_scales size:  torch.Size([1, 500])
index_positions size:  torch.Size([1, 500, 2])
index_global device:  -1
index_local device:  -1
index_mask device:  -1
index_scales device:  -1
index_positions device:  -1
query_global size:  torch.Size([1, 2048])
query_local size:  torch.Size([1, 500, 128])
query_mask size:  torch.Size([1, 500])
query_scales size:  torch.Size([1, 500])
query_positions size:  torch.Size([1, 500, 2])


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 31.75 GiB total capacity; 30.53 GiB already allocated; 4.00 MiB free; 30.60 GiB reserved in total by PyTorch)

In [77]:
current_scores.size()

torch.Size([100])

top_k:  100
query_global size:  torch.Size([100, 2048])
query_local size:  torch.Size([100, 500, 128])
query_mask size:  torch.Size([100, 500])
query_scales size:  torch.Size([100, 500])
query_positions size:  torch.Size([100, 500, 2])


In [74]:
scores = []

for i in tqdm(range(2)):
    nnids = eval_nn_inds[:, i]
    index_global        = []
    index_local         = []
    index_mask          = []
    index_scales        = []
    index_positions     = []
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    for iterator in range(nnids.size(dim=0)):
        index_global.append(gallery_global[iterator, nnids[iterator]])
        index_local.append(gallery_local[iterator, nnids[iterator]])
        index_mask.append(gallery_mask[iterator, nnids[iterator]])
        index_scales.append(gallery_scales[iterator, nnids[iterator]])
        index_positions.append(gallery_positions[iterator, nnids[iterator]])
        
    torch.cuda.empty_cache()
    gc.collect()
    index_global = np.stack(index_global, axis=0)
    index_global = torch.from_numpy(index_global)
    index_global = index_global.to(device)
    
    torch.cuda.empty_cache()
    gc.collect()
    index_local = np.stack(index_local, axis=0)
    index_local = torch.from_numpy(index_local)
    index_local = index_local.to(device)
        
    torch.cuda.empty_cache()
    gc.collect()
    index_mask = np.stack(index_mask, axis=0)
    index_mask = torch.from_numpy(index_mask)
    index_mask = index_mask.to(device)
    
    torch.cuda.empty_cache()
    gc.collect()
    index_scales = np.stack(index_scales, axis=0)
    index_scales = torch.from_numpy(index_scales)
    index_scales = index_scales.to(device)
        
    torch.cuda.empty_cache()
    gc.collect()
    index_positions = np.stack(index_positions, axis=0)
    index_positions = torch.from_numpy(index_positions)
    index_positions = index_positions.to(device)

    print("index_global size: ", index_global.shape)
    print("index_local size: ", index_local.shape)
    print("index_mask size: ", index_mask.shape)
    print("index_scales size: ", index_scales.shape)
    print("index_positions size: ", index_positions.shape)
    
    
    print("index_global device: ", index_global.get_device())
    print("index_local device: ", index_local.get_device())
    print("index_mask device: ", index_mask.get_device())
    print("index_scales device: ", index_scales.get_device())
    print("index_positions device: ", index_positions.get_device())
    
    current_scores = model(
        query_global, query_local, query_mask, query_scales, query_positions,
        index_global,
        index_local,
        index_mask,
        index_scales,
        index_positions)

  0%|          | 0/2 [00:00<?, ?it/s]

index_global size:  torch.Size([100, 2048])
index_local size:  torch.Size([100, 500, 128])
index_mask size:  torch.Size([100, 500])
index_scales size:  torch.Size([100, 500])
index_positions size:  torch.Size([100, 500, 2])
index_global device:  0
index_local device:  0
index_mask device:  0
index_scales device:  0
index_positions device:  0


 50%|█████     | 1/2 [00:00<00:00,  1.03it/s]

index_global size:  torch.Size([100, 2048])
index_local size:  torch.Size([100, 500, 128])
index_mask size:  torch.Size([100, 500])
index_scales size:  torch.Size([100, 500])
index_positions size:  torch.Size([100, 500, 2])
index_global device:  0
index_local device:  0
index_mask device:  0
index_scales device:  0
index_positions device:  0


 50%|█████     | 1/2 [00:01<00:01,  1.86s/it]


RuntimeError: CUDA out of memory. Tried to allocate 394.00 MiB (GPU 0; 31.75 GiB total capacity; 29.92 GiB already allocated; 44.00 MiB free; 30.56 GiB reserved in total by PyTorch)

In [ ]:
index_global    = []
for idx in range(nnids.size(dim=0)):
    index_global.append(gallery_global[idx, nnids[idx]])
index_global = np.stack(index_global, axis=0)
index_global = torch.from_numpy(index_global).cuda("cuda:0")
index_local     = []
for idx in range(nnids.size(dim=0)):
    index_local.append(gallery_local[idx, nnids[idx]])
index_local = np.stack(index_local, axis=0)
index_local = torch.from_numpy(index_local).cuda("cuda:0")
index_mask     = []
for idx in range(nnids.size(dim=0)):
    index_mask.append(gallery_mask[idx, nnids[idx]])
index_mask = np.stack(index_mask, axis=0)
index_mask = torch.from_numpy(index_mask).cuda("cuda:0")
index_scales     = []
for idx in range(nnids.size(dim=0)):
    index_scales.append(gallery_scales[idx, nnids[idx]])
index_scales = np.stack(index_scales, axis=0)
index_scales = torch.from_numpy(index_scales).cuda("cuda:0")
#index_scales    = gallery_scales[:, nnids]
index_positions     = []
for idx in range(nnids.size(dim=0)):
    index_positions.append(gallery_positions[idx, nnids[idx]])
index_positions = np.stack(index_positions, axis=0)
index_positions = torch.from_numpy(index_positions).cuda("cuda:0")
#index_positions = gallery_positions[:, nnids]

In [ ]:
index_global.shape, index_local.shape, index_mask.shape, index_scales.shape, index_positions.shape

In [ ]:
query_global
#query_local, 
#query_mask, 
#query_scales, 
#query_positions

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device

In [ ]:
index_scales.to(device)

In [ ]:
index_global.to(device),
index_local.to(device),
index_mask.to(device),
index_scales.to(device),
index_positions.to(device)

In [ ]:
scores = []
for i in tqdm(range(top_k)):
    nnids = eval_nn_inds[:, i]
    index_global    = []
    for idx in range(nnids.size(dim=0)):
        index_global.append(gallery_global[idx, nnids[idx]])
    index_global = np.stack(index_global, axis=0)
    index_global = torch.from_numpy(index_global)
    index_local     = []
    for idx in range(nnids.size(dim=0)):
        index_local.append(gallery_local[idx, nnids[idx]])
    index_local = np.stack(index_local, axis=0)
    index_local = torch.from_numpy(index_local)
    index_mask     = []
    for idx in range(nnids.size(dim=0)):
        index_mask.append(gallery_mask[idx, nnids[idx]])
    index_mask = np.stack(index_mask, axis=0)
    index_mask = torch.from_numpy(index_mask)
    index_scales     = []
    for idx in range(nnids.size(dim=0)):
        index_scales.append(gallery_scales[idx, nnids[idx]])
    index_scales = np.stack(index_scales, axis=0)
    index_scales = torch.from_numpy(index_scales)
    index_positions     = []
    for idx in range(nnids.size(dim=0)):
        index_positions.append(gallery_positions[idx, nnids[idx]])
    index_positions = np.stack(index_positions, axis=0)
    index_positions = torch.from_numpy(index_positions)
    
    index_global = index_global.to(device)
    index_local = index_local.to(device)
    index_mask = index_mask.to(device)
    index_scales = index_scales.to(device)
    index_positions = index_positions.to(device)

    print("index_global size: ", index_global.shape)
    print("index_local size: ", index_local.shape)
    print("index_mask size: ", index_mask.shape)
    print("index_scales size: ", index_scales.shape)
    print("index_positions size: ", index_positions.shape)
    
    current_scores = model(
        query_global, query_local, query_mask, query_scales, query_positions,
        index_global.to(device),
        index_local.to(device),
        index_mask.to(device),
        index_scales.to(device),
        index_positions.to(device))

In [ ]:
index_global.shape, index_local.shape, index_mask.shape, index_scales.shape, index_positions.shape

In [ ]:
gallery_global[:, nnids].shape

In [ ]:
scores = []
for i in tqdm(range(top_k)):
    print("gallery_global size: ", gallery_global.shape)
    print("gallery_local size: ", gallery_local.shape)
    print("gallery_mask size: ", gallery_mask.shape)
    print("gallery_scales size: ", gallery_scales.shape)
    print("gallery_positions size: ", gallery_positions.shape)
    nnids = eval_nn_inds[:, i]
    index_global    = []
    for idx in range(nnids.size(dim=0)):
        index_global.append(gallery_global[idx, nnids[idx]])
    index_global = np.stack(index_global, axis=0)
    index_global = torch.from_numpy(index_global)
    index_local     = []
    for idx in range(nnids.size(dim=0)):
        index_local.append(gallery_local[idx, nnids[idx]])
    index_local = np.stack(index_local, axis=0)
    index_local = torch.from_numpy(index_local)
    index_mask     = []
    for idx in range(nnids.size(dim=0)):
        index_mask.append(gallery_mask[idx, nnids[idx]])
    index_mask = np.stack(index_mask, axis=0)
    index_mask = torch.from_numpy(index_mask)
    index_scales     = []
    for idx in range(nnids.size(dim=0)):
        index_scales.append(gallery_scales[idx, nnids[idx]])
    index_scales = np.stack(index_scales, axis=0)
    index_scales = torch.from_numpy(index_scales)
    #index_scales    = gallery_scales[:, nnids]
    index_positions     = []
    for idx in range(nnids.size(dim=0)):
        index_positions.append(gallery_positions[idx, nnids[idx]])
    index_positions = np.stack(index_positions, axis=0)
    index_positions = torch.from_numpy(index_positions)
    torch.cuda.empty_cache()
    print("index_global size: ", index_global.shape)
    print("index_local size: ", index_local.shape)
    print("index_mask size: ", index_mask.shape)
    print("index_scales size: ", index_scales.shape)
    print("index_positions size: ", index_positions.shape)
    
    print("index_global device: ", index_global.get_device())
    print("index_local device: ", index_local.get_device())
    print("index_mask device: ", index_mask.get_device())
    print("index_scales device: ", index_scales.get_device())
    print("index_positions device: ", index_positions.get_device())
    
    index_global.to(device),
    index_local.to(device),
    index_mask.to(device),
    index_scales.to(device),
    index_positions.to(device)

    current_scores = model(
        query_global, query_local, query_mask, query_scales, query_positions,
        index_global.to(device),
        index_local.to(device),
        index_mask.to(device),
        index_scales.to(device),
        index_positions.to(device))
    scores.append(current_scores.cpu().data)
scores = torch.stack(scores, -1) # 70 x 100
closest_dists, indices = torch.sort(scores, dim=-1, descending=True)
closest_indices = torch.gather(medium_nn_inds, -1, indices)
ranks = deepcopy(medium_nn_inds)
ranks[:, :top_k] = deepcopy(closest_indices)
ranks = ranks.cpu().data.numpy().T
# pickle_save('medium_nn_inds.pkl', ranks.T)
out = compute_metrics('viquae', ranks, gnd['gnd'], kappas=ks)

In [ ]:
  
    scores = []
    for i in tqdm(range(top_k)):
        nnids = medium_nn_inds[:, i]
        index_global    = gallery_global[nnids]
        index_local     = gallery_local[nnids]
        index_mask      = gallery_mask[nnids]
        index_scales    = gallery_scales[nnids]
        index_positions = gallery_positions[nnids]
        current_scores = model(
            query_global, query_local, query_mask, query_scales, query_positions,
            index_global.to(device),
            index_local.to(device),
            index_mask.to(device),
            index_scales.to(device),
            index_positions.to(device))
        scores.append(current_scores.cpu().data)
    scores = torch.stack(scores, -1) # 70 x 100
    closest_dists, indices = torch.sort(scores, dim=-1, descending=True)
    closest_indices = torch.gather(medium_nn_inds, -1, indices)
    ranks = deepcopy(medium_nn_inds)
    ranks[:, :top_k] = deepcopy(closest_indices)
    ranks = ranks.cpu().data.numpy().T
    # pickle_save('medium_nn_inds.pkl', ranks.T)
    out = compute_metrics('viquae', ranks, gnd['gnd'], kappas=ks)

    ########################################################################################
    ########################################################################################  
    
    return out

In [ ]:
eval_function = partial(evaluate, model=model, 
        cache_nn_inds=cache_nn_inds,
        recall=recall_ks, query_loader=loaders.query, gallery_loader=loaders.gallery)

In [ ]:
#metrics = eval_function()

In [ ]:
#pprint(metrics)

In [ ]:
from utils.data.delf import datum_io

In [ ]:
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True

In [ ]:
with open(osp.join(data_dir,  set_name+'_query.txt')) as fid:
    query_lines   = fid.read().splitlines()

In [ ]:
len(query_lines)

In [ ]:
with open(osp.join(data_dir, set_name+'_gallery.txt')) as fid:
    gallery_lines = fid.read().splitlines()

In [ ]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    query_feats.append(datum_io.ReadFromFile(path))

In [ ]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [ ]:
query_feats.shape

In [ ]:
selection_lines = np.genfromtxt('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/'+
                                set_name+'_selection_imgs.txt', dtype='str')
selection_lines.shape

In [ ]:
selection_lines[0][:10]

In [ ]:
# wiki_img   = '.'.join((wiki_item['image'].split('.')[:-1]))

In [ ]:
selection_index_feats = []
for i in tqdm(range(len(selection_lines))):
    index_feats = []
    for image_file in selection_lines[i]:
        name = '.'.join((image_file.split('.')[:-1]))
        path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
        index_feats.append(datum_io.ReadFromFile(path))
    selection_index_feats.append(index_feats)

In [ ]:
min([len(selection_index_feats[i]) for i in range(len(selection_index_feats))])

In [ ]:
selection_index_feats = np.array(selection_index_feats)
selection_index_feats.shape

In [ ]:
query_feats[0].shape

In [ ]:
selection_sims = []
for i in range(len(selection_index_feats)):
    index_feats = np.stack(selection_index_feats[i], axis=0)
    index_feats = index_feats / LA.norm(index_feats, axis=-1)[:, None]
    selection_sims.append(np.matmul(query_feats[i], index_feats.T))

In [ ]:
sims = np.array(selection_sims)
sims.shape

In [ ]:
sims

In [ ]:
if use_aqe:
    ## WARNING: I WAS TOO LAZY TO CORRECT IT
    ## IF YOU WANNA USE AQE PARAMATER - ADAPT THE CODE FOR VIQUAE-RRT
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)

In [ ]:
selection_index_feats[0].shape

In [ ]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    index_feats = selection_index_feats[i]
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [ ]:
nn_inds.shape

In [ ]:
if save_nn_inds:
    output_path = osp.join(data_dir, set_name + '_nn_inds_%s.pkl' % feature_name)
    pickle_save(output_path, nn_inds)

In [ ]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [ ]:
gnd_data['simlist']

In [ ]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
    print(ranks.shape)
    
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol
    elif dataset.startswith('viquae'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, kappas)


        out = {
            'H_map': np.around(mapH*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
        }

        print('>> {}: mAP H: {}'.format(dataset, out['H_map']))
        print('>> {}: mP@k{} H: {}'.format(dataset, kappas, out['H_mp']))

    return out

In [ ]:
np.arange(nn_inds.T.shape[0])[np.in1d(nn_inds.T[:,i], gnd_data['gnd'][0]['junk'])]

In [ ]:
compute_metrics('viquae', nn_inds.T, gnd_data['gnd'], kappas=[1,5,6,10])